In [118]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
import pickle
import math
import numpy as np
import torch as T

In [2]:
dataset_path = "datasets/MiniGrid-SimpleCrossingS11N5-v0-CrossingS11N5_A2C_Fullgrid-10.pickle"

In [3]:
with open(dataset_path, 'rb') as handle:
    data = pickle.load(handle)

In [9]:
data[9][3]['obs'].keys()

dict_keys(['image', 'direction', 'mission', 'partial_image', 'pixels', 'partial_pixels'])

In [15]:
data[0].keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34])

In [34]:
print([data[0][i]['action'] for i in range(len(data[0]))])

[2, 2, 2, 2, 2, 4, 5, 6, 2, 5, 5, 1, 2, 5, 4, 2, 3, 0, 2, 1, 2, 5, 2, 5, 2, 2, 0, 4, 2, 2, 1, 2, 3, 3, 2]


In [38]:
data[0][11]['direction']

0

In [39]:
data[0][12]['direction']

1

In [50]:
np.allclose(data[0][11]['obs']['partial_image'], data[0][12]['obs']['partial_image'])

False

In [55]:
data[0][11]['obs']['image'][:,:,0]

array([[ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
       [ 2,  1,  2,  1,  2,  1,  1,  1,  2,  1,  2],
       [ 2,  1,  2,  1,  2,  1,  1,  1,  2,  1,  2],
       [ 2,  1,  2,  1,  2,  1,  1,  1,  2,  1,  2],
       [ 2,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2],
       [ 2,  1,  2,  1,  2,  1,  1,  1,  2,  1,  2],
       [ 2,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2],
       [ 2, 10,  1,  1,  2,  1,  1,  1,  2,  1,  2],
       [ 2,  1,  2,  1,  1,  1,  1,  1,  2,  1,  2],
       [ 2,  1,  2,  1,  2,  1,  1,  1,  1,  8,  2],
       [ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2]], dtype=uint8)

In [60]:
 data[0][12]['obs']['image'][:,:,0]

array([[ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
       [ 2,  1,  2,  1,  2,  1,  1,  1,  2,  1,  2],
       [ 2,  1,  2,  1,  2,  1,  1,  1,  2,  1,  2],
       [ 2,  1,  2,  1,  2,  1,  1,  1,  2,  1,  2],
       [ 2,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2],
       [ 2,  1,  2,  1,  2,  1,  1,  1,  2,  1,  2],
       [ 2,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2],
       [ 2, 10,  1,  1,  2,  1,  1,  1,  2,  1,  2],
       [ 2,  1,  2,  1,  1,  1,  1,  1,  2,  1,  2],
       [ 2,  1,  2,  1,  2,  1,  1,  1,  1,  8,  2],
       [ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2]], dtype=uint8)

In [58]:
 data[0][12]['location']

array([7, 2])

In [86]:
[data[0][j]['direction'] for j in range(len(data[0]))]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1]

In [89]:
T.Tensor([data[episode][step]['direction'] for episode in range(len(data)) for step in range(len(data[episode]))]).shape

torch.Size([481])

In [90]:
[episode for episode in range(len(data)) for step in range(len(data[episode]))]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,


In [141]:
support_trajectories = T.Tensor(np.array([data[episode][step]['obs']['partial_image'] for episode in range(len(data)) for step in range(len(data[episode]))]).transpose(0,3,1,2))
support_targets = T.tensor(np.array([episode for episode in range(len(data)) for step in range(len(data[episode]))]))

In [155]:
from generative_contrastive_modelling.gcm_encoder import GCMEncoder
from generative_contrastive_modelling.gcm import GenerativeContrastiveModelling

GCM = GenerativeContrastiveModelling(support_trajectories.shape, 64)
gcm_encoder = GCMEncoder(support_trajectories.shape, 64)

In [127]:
gcm_encoder.forward(support_trajectories)

(tensor([[-0.0385,  0.2495, -0.2756,  ..., -0.2030,  0.5135,  0.3631],
         [-0.1600, -0.0145,  0.4571,  ...,  0.0618, -0.4870,  0.1450],
         [ 0.2745,  0.0623, -0.4905,  ..., -0.2901,  0.0526,  0.3152],
         ...,
         [ 0.3522, -0.1163,  0.3499,  ...,  0.0660,  0.1318,  0.2182],
         [ 0.3522, -0.1163,  0.3499,  ...,  0.0660,  0.1318,  0.2182],
         [ 0.3522, -0.1163,  0.3499,  ...,  0.0660,  0.1318,  0.2182]],
        grad_fn=<AddmmBackward0>),
 tensor([[0.7433, 1.3091, 0.8501,  ..., 1.2642, 1.8275, 0.4451],
         [0.9667, 0.6985, 1.0885,  ..., 1.0073, 1.8996, 1.2991],
         [0.6674, 0.9682, 0.6860,  ..., 0.7043, 2.3179, 0.8435],
         ...,
         [0.9340, 0.6966, 0.5456,  ..., 1.0280, 1.0395, 0.3358],
         [0.9340, 0.6966, 0.5456,  ..., 1.0280, 1.0395, 0.3358],
         [0.9340, 0.6966, 0.5456,  ..., 1.0280, 1.0395, 0.3358]],
        grad_fn=<ExpBackward0>))

In [107]:
support_trajectories.shape

torch.Size([481, 3, 7, 7])

In [180]:
indices = T.randperm(support_trajectories.shape[0])[:5]
query_observations = support_trajectories[indices]
query_targets = support_targets[indices]

In [181]:
query_targets

tensor([8, 9, 2, 7, 7])

In [182]:
observations=T.cat([support_trajectories, query_observations], dim=0)

In [183]:
GCM.compute_loss(support_trajectories=support_trajectories, support_targets=support_targets, query_observations=query_observations, query_targets=query_targets)

torch.Size([1, 10])
torch.Size([1, 481])
torch.Size([1, 481])
torch.Size([1, 10, 5])
torch.Size([1, 5])


{'predictions': tensor([[5, 9, 2, 3, 8]]),
 'loss': tensor(2.3013, grad_fn=<NllLoss2DBackward0>)}

In [176]:
support_targets.squeeze_(0).shape

torch.Size([481])

In [177]:
support_targets.shape

torch.Size([481])